# Udacity Data Engineering Capstone

# Table of contents
1. [Imports](#imports)
2. [Step 1: Project Scope and Data Gathering](#step1)
    * [Scope](#scope)
    * [Data Description](#data_desc)
3. [Step 2: Data Exploration & Modeling](#step2)
4. [Step 3: Define the Data Model](#step3)
    * [3.1 Conceptual Data Model](#data_model)
    * [3.2 Mapping Out Data Pipelines](#pipeline_steps)
5. [Step 4: Run Pipelines to Model the Data](#step4)
6. [Step 5: Complete Project Write Up](#step5)

## Imports <a name="imports"></a>

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [3]:
import seaborn as sns

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, rand
from pyspark.sql.functions import isnan, when, count, col

In [5]:
import configparser
import psycopg2

## Step 1: Project Scope and Data Gathering <a name="step1"></a>

### Scope <a name="scope"></a>

For my capstone project I developed a data pipeline that creates an analytics database for querying information about immigration into the U.S. The analytics tables are hosted in a Redshift Database and the pipeline implementation was done using Apache Airflow.

### Data Description <a name="data_desc"></a>

The following datasets were used to create the analytics database:
* I94 Immigration Data: This data comes from the US National Tourism and Trade Office found [here](https://travel.trade.gov/research/reports/i94/historical/2016.html). Each report contains international visitor arrival statistics by world regions and select countries (including top 20), type of visa, mode of transportation, age groups, states visited (first intended address only), and the top ports of entry (for select countries).
* World Temperature Data: This dataset came from Kaggle found [here](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).
* U.S. City Demographic Data: This dataset contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000. Dataset comes from OpenSoft found [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).
* Airport Code Table: This is a simple table of airport codes and corresponding cities. The airport codes may refer to either IATA airport code, a three-letter code which is used in passenger reservation, ticketing and baggage-handling systems, or the ICAO airport code which is a four letter code used by ATC systems and for airports that do not have an IATA airport code (from wikipedia). It comes from [here](https://datahub.io/core/airport-codes#data).

#### I94 Immigration Data pull

In [6]:
imm_data = pd.read_csv('data/immigration_data_sample.csv', index_col='cicid')
print(imm_data.shape[0])
imm_data.head()

1000


,Unnamed: 0,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
cicid,,,,,,,,,,,,,,,,,,,,,
4084316.0,2027561,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
4422636.0,2171295,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
1195600.0,589494,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
5291768.0,2631158,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
985523.0,3032257,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [ ]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [ ]:
imm_data_spark = spark.read.parquet("sas_data")
print(imm_data_spark.count())
imm_data_spark.limit(10).toPandas()

#### U.S. City Demographic Data pull

In [ ]:
city_dem_data = pd.read_csv('data/us-cities-demographics.csv', sep=';')
print(city_dem_data.info())
city_dem_data.head()

#### Airport Code Data

In [ ]:
airport_code_data = pd.read_csv('data/airport-codes_csv.csv')
print(airport_code_data.info())
airport_code_data.head()

#### World Temperature Data

In [ ]:
temp_data = pd.read_csv('data/GlobalLandTemperaturesByCity.csv')
print(temp_data.info())
temp_data.head()

## Step 2: Data Exploration & Modeling <a name="step2"></a>

### Data Prep

In [ ]:
imm_data_spark.where(imm_data_spark['dtaddto'].isNotNull().alias('dtaddto')).select(F.count('dtaddto')).collect()

In [ ]:
def sas_program_file_value_parser(sas_source_file, value, columns):
    """Parses SAS Program file to return value as pandas dataframe
    Args:
        sas_source_file (str): SAS source code file.
        value (str): sas value to extract.
        columns (list): list of 2 containing column names.
    Return:
        None
    """
    file_string = ''
    
    with open(sas_source_file) as f:
        file_string = f.read()
    
    file_string = file_string[file_string.index(value):]
    file_string = file_string[:file_string.index(';')]
    
    line_list = file_string.split('\n')[1:]
    codes = []
    values = []
    
    for line in line_list:
        
        if '=' in line:
            code, val = line.split('=')
        
            codes.append(code.strip())
            values.append(val.strip())
        
            
    return pd.DataFrame(zip(codes,values), columns=columns)

In [ ]:
i94cit_res = sas_program_file_value_parser('data/I94_SAS_Labels_Descriptions.SAS', 'i94cntyl', ['code', 'country'])
i94cit_res.head()

In [ ]:
config = configparser.ConfigParser()
config.read('dwh.cfg')
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}"\
            .format(*config['CLUSTER'].values()))

In [ ]:
i94port = sas_program_file_value_parser('data/I94_SAS_Labels_Descriptions.SAS', 'i94prtl', ['code', 'port'])
i94port.head()

In [ ]:
i94mode = sas_program_file_value_parser('data/I94_SAS_Labels_Descriptions.SAS', 'i94model', ['code', 'mode'])
i94mode.head()

In [ ]:
conn.get_dsn_parameters()

In [ ]:
i94cit_res.to_sql('i94cit_res', conn, index=False, if_exists='replace')

In [ ]:
i94addr = sas_program_file_value_parser('data/I94_SAS_Labels_Descriptions.SAS', 'i94addrl', ['code', 'addr'])
i94addr.head()

In [ ]:
i94visa = sas_program_file_value_parser('data/I94_SAS_Labels_Descriptions.SAS', 'I94VISA', ['code', 'type'])
i94visa.head()

#### I94 Immigration Data prep

In [ ]:
imm_data_spark.printSchema()

In [ ]:
int_columns = ['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94mode', 'i94bir', 'i94visa', 'count', 'biryear']
for col in int_columns:
    imm_data_spark = imm_data_spark.withColumn(col, imm_data_spark[col].cast('int'))

In [ ]:
for col in int_columns:
    if imm_data_spark.select(col).orderBy(rand()).limit(10).collect():
        pass

In [ ]:
imm_data_spark.select('dtaddto').orderBy(rand()).limit(10).collect()

In [ ]:
datetime.strptime('10252016', '%m%d%Y').date()

In [ ]:
date_from_sas = udf(lambda x: pd.to_timedelta(x, unit='D')+pd.Timestamp('1960-1-1').date(), DateType())

In [ ]:
pd.to_timedelta(19411.0, unit='D')+ pd.Timestamp('1960-1-1').date()

In [ ]:
sas_date_columns = ['arrdate', 'depdate']
for col in sas_date_columns:
    imm_data_spark = imm_data_spark.withColumn(col, date_from_sas(imm_data_spark[col]))

In [ ]:
x = '01252016'
date(int(x[4:]), int(x[:2]), int(x[2:4]))

In [ ]:
# date_from_str = udf(lambda x: datetime.strptime(x, '%m%d%Y').date(), DateType())
from datetime import date
date_from_str = udf(lambda x: date(int(x[4:]), int(x[:2]), int(x[2:4])) 
                    if (len(x)==8 and int(x[:2]) <= 12 and int(x[:2]) >- 1) else None, DateType())

In [ ]:
imm_data_spark.filter(imm_data_spark['dtadfile'].isNull()).count()

In [ ]:
imm_data_spark.withColumn('dtadfile', date_from_str(imm_data_spark['dtadfile'])).limit(10).collect()

In [ ]:
# str_date_columns = ['dtadfile', 'dtaddto']
# for col in str_date_columns:
#     imm_data_spark = imm_data_spark.withColumn(col, date_from_str(imm_data_spark[col]))

In [ ]:
imm_data_spark.printSchema()

## Step 3: Define the Data Model <a name="step3"></a>

### 3.1 Conceptual Data Model <a name="data_model"></a>

Map out the conceptual data model and explain why you chose that model

### 3.2 Mapping Out Data Pipelines <a name="pipeline_steps"></a>

Steps necessary to pipeline the data into the chosen data model:


    >> Begin Dummy Operator.

        >> Operator extract tables from I94 labels mappings files and stage to S3/local as csv:
            * i94cit_res
            * i94port
            * i94mode
            * i94addr
            * i94visa   
            >> Copy the above csv files from local/s3 to create tables in Redshift.
                >> Perform data quality checks for the tables above.
            >> Transform immigration data files on local/s3 and write results to `immigration` Redshift table.
                >> Perform data qualitiy checks for immigration table

        >> Copy csv files from local/s3 to create the following tables in Redshift.
            * us_cities_demographics
            * airport_codes
            * world_temperature
            >> Perform data quality checks on above tables.
            
                >> End Dummy Operator.
  

## Step 4: Run Pipelines to Model the Data <a name="step4"></a>

### 4.1 Create the data model

Build the data pipelines to create the data model.

Create Tables:
```bash
(venv)$ create_tables.py
```

Launch Airflow UI:
1. Initialize Airflow & Run Webserver
```bash
(venv) $ export AIRFLOW_HOME=$(pwd)
(venv) $ airflow initdb
(venv) $ airflow webserver -p 8080
```
2. Run Scheduler (Open New Terminal Tab)
```bash
(venv) $ export AIRFLOW_HOME=$(pwd)
(venv) $ airflow scheduler
```
3. Access Airflow UI at `localhost:8080`
4. Run `etl_dag` in Airflow UI

## Step 5: Complete Project Write Up <a name="step5"></a>

### Technology Choices and tools

* Clearly state the rationale for the choice of tools and technologies for the project.

1. Apache Airflow: Allows for easy scheduling and monitoring etl workflows for keeping analytics database up to date
2. PySpark: Allows to load large immigration dataset and perform data prep and/or analysis before writing to redshift table

### Data Schedule Proposal

* Propose how often the data should be updated and why.

Pipeline will be scheduled monthly as immigration data is the primary datasource is on a monthly granularity

### Possible Scenerios, changes and approach

* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.